## Importar librerías
   

In [1]:
import pandas as pd
import numpy as np
import os
import data_util as util

## Cargar datos


In [2]:
ruta = '../data/aire/'
ruta_nuevoCSV = '../data/auxiliar/aire_2022.csv'
file_names = os.listdir(ruta)
file_names

['abr_mo22.csv',
 'ago_mo22.csv',
 'dic_mo22.csv',
 'ene_mo22.csv',
 'feb_mo22.csv',
 'jul_mo22.csv',
 'jun_mo22.csv',
 'mar_mo22.csv',
 'may_mo22.csv',
 'nov_mo22.csv',
 'oct_mo22.csv',
 'sep_mo22.csv']

## Crear único dataset con los los datos del año 2022
Recorrer la lista files_names para crear un único dataset


In [3]:
df_aire = pd.DataFrame()
for file_name in file_names:
    df_current = pd.read_csv(ruta + file_name, delimiter=';')
    print(file_name, df_current.shape)
    df_aire = pd.concat([df_aire, df_current], ignore_index=True)
df_aire.head(3)

abr_mo22.csv (3951, 56)
ago_mo22.csv (3960, 56)
dic_mo22.csv (4092, 56)
ene_mo22.csv (3929, 56)
feb_mo22.csv (3556, 56)
jul_mo22.csv (3968, 56)
jun_mo22.csv (3872, 56)
mar_mo22.csv (3946, 56)
may_mo22.csv (4059, 56)
nov_mo22.csv (3960, 56)
oct_mo22.csv (4092, 56)
sep_mo22.csv (3838, 56)


,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2022,4,1,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V
1,28,79,4,1,28079004_1_38,2022,4,2,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V
2,28,79,4,1,28079004_1_38,2022,4,3,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V


## Seleccionar datos

Nos quedamos con los parámetros que nos interesan para el análisis: 
* SO2: Dióxido de Azufre
* CO: Monóxido de Carbono
* NO2: Dióxido de Nitrógeno
* PM2.5: Partículas < 2.5 μm
* PM10: Partículas < 10 μm
* O3: Ozono
* TOL: Tolueno
* BEN: Benceno
* EBE: Etilbenceno

In [4]:
df_aire = df_aire[(df_aire['MAGNITUD'] == 1) | (df_aire['MAGNITUD'] == 6) | (df_aire['MAGNITUD'] == 8) 
                | (df_aire['MAGNITUD'] == 9) | (df_aire['MAGNITUD'] == 10) | (df_aire['MAGNITUD'] == 14) 
                | (df_aire['MAGNITUD'] == 20) | (df_aire['MAGNITUD'] == 30)| (df_aire['MAGNITUD'] == 35)]
df_aire.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2022,4,1,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V
1,28,79,4,1,28079004_1_38,2022,4,2,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V
2,28,79,4,1,28079004_1_38,2022,4,3,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,1.0,V
3,28,79,4,1,28079004_1_38,2022,4,4,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
4,28,79,4,1,28079004_1_38,2022,4,5,1.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V


## Selección de estaciones de medida

Se selecciona la estacion de medida de código_corto = 8, porque es la única que mide todos los contaminantes atmosféricos que son necesarios para calcular el índice de calidad del aire (ICA)

In [5]:
df_aire  = df_aire[(df_aire['ESTACION'] == 8)]
df_aire.head(2)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
150,28,79,8,1,28079008_1_38,2022,4,1,4.0,V,...,4.0,V,4.0,V,4.0,V,4.0,V,4.0,V
151,28,79,8,1,28079008_1_38,2022,4,2,4.0,V,...,5.0,V,4.0,V,4.0,V,4.0,V,4.0,V


## Crear una nueva columna fecha
Con formato datetime

In [6]:
df_aire['FECHA'] = pd.to_datetime(df_aire[['ANO','MES','DIA']].rename(columns = {'ANO': 'YEAR', 'MES': 'MONTH', 'DIA': 'DAY'}))

## Preparación de los datos de salida:
* Extraer los datos de las medidas tomadas
* Crear dataset auxiliar con los datos extraídos y las nuevas   columnas creadas.

In [7]:
data_columns = [ 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24'] # feature
veri_columns = [ 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']

hora_column = np.arange(1,25) # feature label

df = pd.DataFrame(columns=['HORA', 'DATO', 'VERIFICADO'])
aux = pd.DataFrame(columns=['ANO', 'MES', 'DIA', 'HORA', 'DATO', 'VERIFICADO', 'ESTACION']) # aux de próxima fila a ser añadida
for  (h,d,v) in zip(hora_column, data_columns, veri_columns): 
    aux
    aux['DATO'] = df_aire [ data_columns[h-1] ]
    # aux[['CODIGO', 'MAGNITUD_IDX', 'TECNICA']] = df_aire['PUNTO_MUESTREO'].str.split('_', n=2, expand=True).astype(int)
    aux['VERIFICADO'] = df_aire [ veri_columns[h-1] ] 
    aux['HORA'] = h-1
    aux[['ANO', 'MES', 'DIA']] = df_aire[['ANO','MES','DIA']]
    aux['MAGNITUD'] = df_aire ['MAGNITUD'].astype(int)
    aux['ESTACION'] = df_aire ['ESTACION']
    df = pd.concat([df, aux], ignore_index=True)
df = df.astype({'HORA': 'int32'}, copy = False)
print(df.shape)
df.head()


(78840, 8)


,HORA,DATO,VERIFICADO,ANO,MES,DIA,ESTACION,MAGNITUD
0,0,4.0,V,2022.0,4.0,1.0,8.0,1.0
1,0,4.0,V,2022.0,4.0,2.0,8.0,1.0
2,0,4.0,V,2022.0,4.0,3.0,8.0,1.0
3,0,4.0,V,2022.0,4.0,4.0,8.0,1.0
4,0,4.0,V,2022.0,4.0,5.0,8.0,1.0


## Añadir la columna fecha en formato datetime al nuevo dataset 

In [8]:
df['FECHA'] = pd.to_datetime(df[['ANO','MES','DIA','HORA']].rename(columns = {'ANO': 'year', 'MES': 'month','DIA': 'day', 'HORA': 'hour'}))
df.head(2)

,HORA,DATO,VERIFICADO,ANO,MES,DIA,ESTACION,MAGNITUD,FECHA
0,0,4.0,V,2022.0,4.0,1.0,8.0,1.0,2022-04-01
1,0,4.0,V,2022.0,4.0,2.0,8.0,1.0,2022-04-02


## Ordenar los datos por orden cronológico

In [9]:
df = df.sort_values(['FECHA','HORA'])
df.head(2)

,HORA,DATO,VERIFICADO,ANO,MES,DIA,ESTACION,MAGNITUD,FECHA
828,0,4.0,V,2022.0,1.0,1.0,8.0,1.0,2022-01-01
859,0,0.4,V,2022.0,1.0,1.0,8.0,6.0,2022-01-01


## Marcar datos no válidos en el DataFrame:
* Datos no verificados

In [10]:
df.loc[df['VERIFICADO'] != 'V', ['DATO']] = np.nan
df.loc[df['VERIFICADO'] != 'V']

,HORA,DATO,VERIFICADO,ANO,MES,DIA,ESTACION,MAGNITUD,FECHA
60144,18,NaN,N,2022.0,1.0,1.0,8.0,20.0,2022-01-01 18:00:00
63429,19,NaN,N,2022.0,1.0,1.0,8.0,20.0,2022-01-01 19:00:00
66714,20,NaN,N,2022.0,1.0,1.0,8.0,20.0,2022-01-01 20:00:00
69999,21,NaN,N,2022.0,1.0,1.0,8.0,20.0,2022-01-01 21:00:00
73284,22,NaN,N,2022.0,1.0,1.0,8.0,20.0,2022-01-01 22:00:00
...,...,...,...,...,...,...,...,...,...
49976,15,NaN,N,2022.0,12.0,29.0,8.0,10.0,2022-12-29 15:00:00
53230,16,NaN,N,2022.0,12.0,29.0,8.0,9.0,2022-12-29 16:00:00
53261,16,NaN,N,2022.0,12.0,29.0,8.0,10.0,2022-12-29 16:00:00
56515,17,NaN,N,2022.0,12.0,29.0,8.0,9.0,2022-12-29 17:00:00


## Borrar columna verificado

In [11]:
df = df.drop('VERIFICADO', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78840 entries, 828 to 76382
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   HORA      78840 non-null  int32         
 1   DATO      75091 non-null  float64       
 2   ANO       78840 non-null  float64       
 3   MES       78840 non-null  float64       
 4   DIA       78840 non-null  float64       
 5   ESTACION  78840 non-null  float64       
 6   MAGNITUD  78840 non-null  float64       
 7   FECHA     78840 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int32(1)
memory usage: 5.1 MB


## Añadir columnas que faltan y formatearlas,

In [12]:
df = df.set_index(['FECHA', 'ANO', 'MES', 'DIA', 'HORA', 'ESTACION','MAGNITUD'])['DATO'].unstack(level=-1).reset_index().rename_axis(None)
df.head()

MAGNITUD,FECHA,ANO,MES,DIA,HORA,ESTACION,1.0,6.0,8.0,9.0,10.0,14.0,20.0,30.0,35.0
0,2022-01-01 00:00:00,2022.0,1.0,1.0,0,8.0,4.0,0.4,35.0,15.0,16.0,18.39,2.1,0.5,0.3
1,2022-01-01 01:00:00,2022.0,1.0,1.0,1,8.0,4.0,0.4,48.0,25.0,27.0,8.18,2.0,0.5,0.2
2,2022-01-01 02:00:00,2022.0,1.0,1.0,2,8.0,3.0,0.4,47.0,25.0,27.0,7.05,1.7,0.4,0.2
3,2022-01-01 03:00:00,2022.0,1.0,1.0,3,8.0,3.0,0.3,38.0,15.0,17.0,14.22,1.7,0.4,0.1
4,2022-01-01 04:00:00,2022.0,1.0,1.0,4,8.0,3.0,0.3,26.0,3.0,5.0,24.66,1.2,0.4,0.1


## Renombrar columnas con los contaminantes

In [13]:
df = df.rename(columns={1: 'SO2', 6: 'CO', 8: 'NO2', 9: 'PM2.5', 10: 'PM10', 14: 'O3', 20: 'TOL', 30: 'BEN', 35: 'EBE'}, 
                copy = False, errors = 'raise')
df.head()

MAGNITUD,FECHA,ANO,MES,DIA,HORA,ESTACION,SO2,CO,NO2,PM2.5,PM10,O3,TOL,BEN,EBE
0,2022-01-01 00:00:00,2022.0,1.0,1.0,0,8.0,4.0,0.4,35.0,15.0,16.0,18.39,2.1,0.5,0.3
1,2022-01-01 01:00:00,2022.0,1.0,1.0,1,8.0,4.0,0.4,48.0,25.0,27.0,8.18,2.0,0.5,0.2
2,2022-01-01 02:00:00,2022.0,1.0,1.0,2,8.0,3.0,0.4,47.0,25.0,27.0,7.05,1.7,0.4,0.2
3,2022-01-01 03:00:00,2022.0,1.0,1.0,3,8.0,3.0,0.3,38.0,15.0,17.0,14.22,1.7,0.4,0.1
4,2022-01-01 04:00:00,2022.0,1.0,1.0,4,8.0,3.0,0.3,26.0,3.0,5.0,24.66,1.2,0.4,0.1


## Crear columna BTX

* Crear la columna BTX a partir de las columnas TOL, BEN, EBE 
+ Eliminar las columnas: TOL, BEN, EBE

In [14]:
df['BTX'] = df[['TOL', 'BEN', 'EBE']].sum(axis=1)
df.head(2)

MAGNITUD,FECHA,ANO,MES,DIA,HORA,ESTACION,SO2,CO,NO2,PM2.5,PM10,O3,TOL,BEN,EBE,BTX
0,2022-01-01 00:00:00,2022.0,1.0,1.0,0,8.0,4.0,0.4,35.0,15.0,16.0,18.39,2.1,0.5,0.3,2.9
1,2022-01-01 01:00:00,2022.0,1.0,1.0,1,8.0,4.0,0.4,48.0,25.0,27.0,8.18,2.0,0.5,0.2,2.7


In [15]:
df.drop(['TOL', 'BEN', 'EBE'], axis=1, inplace=True)
df.head(1)

MAGNITUD,FECHA,ANO,MES,DIA,HORA,ESTACION,SO2,CO,NO2,PM2.5,PM10,O3,BTX
0,2022-01-01,2022.0,1.0,1.0,0,8.0,4.0,0.4,35.0,15.0,16.0,18.39,2.9


## Añadir columna para el día de la semana

In [16]:
dates = df['FECHA'].dt.to_pydatetime()
days_of_week = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
df['DIA_SEMANA'] = np.array([days_of_week[d.weekday()] for d in dates])
df.head()

MAGNITUD,FECHA,ANO,MES,DIA,HORA,ESTACION,SO2,CO,NO2,PM2.5,PM10,O3,BTX,DIA_SEMANA
0,2022-01-01 00:00:00,2022.0,1.0,1.0,0,8.0,4.0,0.4,35.0,15.0,16.0,18.39,2.9,Sabado
1,2022-01-01 01:00:00,2022.0,1.0,1.0,1,8.0,4.0,0.4,48.0,25.0,27.0,8.18,2.7,Sabado
2,2022-01-01 02:00:00,2022.0,1.0,1.0,2,8.0,3.0,0.4,47.0,25.0,27.0,7.05,2.3,Sabado
3,2022-01-01 03:00:00,2022.0,1.0,1.0,3,8.0,3.0,0.3,38.0,15.0,17.0,14.22,2.2,Sabado
4,2022-01-01 04:00:00,2022.0,1.0,1.0,4,8.0,3.0,0.3,26.0,3.0,5.0,24.66,1.7,Sabado


In [17]:
dia_semana = df.pop('DIA_SEMANA')
df.insert(5, 'DIA_SEMANA', dia_semana)
df.head()

MAGNITUD,FECHA,ANO,MES,DIA,HORA,DIA_SEMANA,ESTACION,SO2,CO,NO2,PM2.5,PM10,O3,BTX
0,2022-01-01 00:00:00,2022.0,1.0,1.0,0,Sabado,8.0,4.0,0.4,35.0,15.0,16.0,18.39,2.9
1,2022-01-01 01:00:00,2022.0,1.0,1.0,1,Sabado,8.0,4.0,0.4,48.0,25.0,27.0,8.18,2.7
2,2022-01-01 02:00:00,2022.0,1.0,1.0,2,Sabado,8.0,3.0,0.4,47.0,25.0,27.0,7.05,2.3
3,2022-01-01 03:00:00,2022.0,1.0,1.0,3,Sabado,8.0,3.0,0.3,38.0,15.0,17.0,14.22,2.2
4,2022-01-01 04:00:00,2022.0,1.0,1.0,4,Sabado,8.0,3.0,0.3,26.0,3.0,5.0,24.66,1.7


In [18]:
df.reset_index(drop=True)
df.head()

MAGNITUD,FECHA,ANO,MES,DIA,HORA,DIA_SEMANA,ESTACION,SO2,CO,NO2,PM2.5,PM10,O3,BTX
0,2022-01-01 00:00:00,2022.0,1.0,1.0,0,Sabado,8.0,4.0,0.4,35.0,15.0,16.0,18.39,2.9
1,2022-01-01 01:00:00,2022.0,1.0,1.0,1,Sabado,8.0,4.0,0.4,48.0,25.0,27.0,8.18,2.7
2,2022-01-01 02:00:00,2022.0,1.0,1.0,2,Sabado,8.0,3.0,0.4,47.0,25.0,27.0,7.05,2.3
3,2022-01-01 03:00:00,2022.0,1.0,1.0,3,Sabado,8.0,3.0,0.3,38.0,15.0,17.0,14.22,2.2
4,2022-01-01 04:00:00,2022.0,1.0,1.0,4,Sabado,8.0,3.0,0.3,26.0,3.0,5.0,24.66,1.7


## Datos no válidos

In [19]:
df.isnull().sum()

MAGNITUD
FECHA            0
ANO              0
MES              0
DIA              0
HORA             0
DIA_SEMANA       0
ESTACION         0
SO2             91
CO              89
NO2             68
PM2.5         1242
PM10          1257
O3             197
BTX              0
dtype: int64

In [20]:
# Porcentaje de datos faltantes
util.calc_missing(df)

SO2 91 / 8760= 1.04 %
CO 89 / 8760= 1.02 %
NO2 68 / 8760= 0.78 %
PM2.5 1242 / 8760= 14.18 %
PM10 1257 / 8760= 14.35 %
O3 197 / 8760= 2.25 %


##  Salvar los datos preprocesados

Se salvan los datos tratados para su posterior fusión con otros csv.

In [21]:
df.to_csv(ruta_nuevoCSV, index=False)